In [1]:
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [2]:
starting_directory = '.'

In [10]:
train_a = pd.read_parquet(f'{starting_directory}/A/train_targets.parquet')
train_b = pd.read_parquet(f'{starting_directory}/B/train_targets.parquet')
train_c = pd.read_parquet(f'{starting_directory}/C/train_targets.parquet')

In [21]:
X_train_estimated_a = pd.read_parquet(f'{starting_directory}/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet(f'{starting_directory}/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet(f'{starting_directory}/C/X_train_estimated.parquet')

X_train_observed_a = pd.read_parquet(f'{starting_directory}/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet(f'{starting_directory}/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet(f'{starting_directory}/C/X_train_observed.parquet')

X_test_estimated_a = pd.read_parquet(f'{starting_directory}/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet(f'{starting_directory}/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet(f'{starting_directory}/C/X_test_estimated.parquet')

In [22]:
def quartersToHours(data):
    hourly_data = []
    for df in data:    
        df["year"] = df['date_forecast'].dt.year
        df["month"] = df['date_forecast'].dt.month
        df["day"] = df['date_forecast'].dt.day
        df["hour"] = df['date_forecast'].dt.hour
    
        group = df.groupby([df["year"], df["month"], df["day"], df["hour"]])  
        result = group.mean()
        result = result.reset_index()
    
        return_df = result.drop(['year', 'month', 'day', 'hour'], axis=1)
        
        hourly_data.append(return_df)

    return hourly_data


def appendLabels(train, val, labels):
    # Rename columns names to match
    val = val.assign(date_forecast=val.date_forecast.dt.round('H'))
    train = train.assign(date_forecast=train.date_forecast.dt.round('H'))

    labels = labels.rename(columns={'time': 'date_forecast'})
    
    joined_train = pd.merge(train, labels, how="inner", on="date_forecast")
    joined_val = pd.merge(val, labels, how="inner", on="date_forecast")

    return [joined_train, joined_val]

def appendBuilding(data, label):
    new_data = []
    for df in data:
        df["building"] = label
        new_data.append(df)
    return new_data

def removeColons(data):
    new_data = []
    for df in data:
        df.columns = df.columns.str.replace(':', '')
        df.columns = df.columns.str.replace('_', '')
        new_data.append(df)
    return new_data
            

In [23]:
A = [X_train_observed_a, X_train_estimated_a, X_test_estimated_a]
B = [X_train_observed_b, X_train_estimated_b, X_test_estimated_b]
C = [X_train_observed_c, X_train_estimated_c, X_test_estimated_c]

A = quartersToHours(A)
B = quartersToHours(B)
C = quartersToHours(C)

A[:2] = appendLabels(A[0], A[1], train_a)
B[:2] = appendLabels(B[0], B[1], train_b)
C[:2] = appendLabels(C[0], C[1], train_c)

A = appendBuilding(A, "A")
B = appendBuilding(B, "B")
C = appendBuilding(C, "C")

A = removeColons(A)
B = removeColons(B)
C = removeColons(C)

In [24]:
A[0].to_csv("csv/X_train_observed_a.csv", index=False)
B[0].to_csv("csv/X_train_observed_b.csv", index=False)
C[0].to_csv("csv/X_train_observed_c.csv", index=False)

A[1].to_csv("csv/X_train_estimated_a.csv", index=False)
B[1].to_csv("csv/X_train_estimated_b.csv", index=False)
C[1].to_csv("csv/X_train_estimated_c.csv", index=False)

A[2].to_csv("csv/X_test_estimated_a.csv", index=False)
B[2].to_csv("csv/X_test_estimated_b.csv", index=False)
C[2].to_csv("csv/X_test_estimated_c.csv", index=False)